In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class TransformerEncoderBlock(nn.Module):
    def __init__(self, emb_dim, ffn_dim, n_heads, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(emb_dim, n_heads)
        self.ln1 = nn.LayerNorm(emb_dim)
        self.ln2 = nn.LayerNorm(emb_dim)

        self.ffn = nn.Sequential(
            nn.Linear(emb_dim, ffn_dim),
            nn.ReLU(),
            nn.Linear(ffn_dim, emb_dim),
            nn.Dropout(dropout),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        self_attn_output = self.self_attn(x, kv=None, mask=mask)
        x = x + self.dropout(self_attn_output) # residual connection
        x = self.ln1(x)
        ffn_output = self.ffn(x)
        x = x + self.dropout(ffn_output)
        x = self.ln2(x)
        return x    

class TransformerDecoderBlock(nn.Module):
    def __init__(self, emb_dim, ffn_dim, n_heads, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(emb_dim)
        self.ln2 = nn.LayerNorm(emb_dim)
        self.ln3 = nn.LayerNorm(emb_dim)
        
        self.self_attn  = MultiHeadAttention(emb_dim, n_heads)  # decoder self-attn
        self.cross_attn = MultiHeadAttention(emb_dim, n_heads)  # encoder-decoder attn
        
        self.ffn = nn.Sequential(
            nn.Linear(emb_dim, ffn_dim),
            nn.ReLU(),
            nn.Linear(ffn_dim, emb_dim),
            nn.Dropout(dropout),
        )
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, encoder_output, self_mask=None, encoder_decoder_mask=None):
        self_attn_output = self.self_attn(x, kv=None, mask=self_mask)
        x = x + self.dropout(self_attn_output)
        x = self.ln1(x)

        encoder_decoder_attn_output = self.cross_attn(q=x, kv=encoder_output, mask=encoder_decoder_mask)
        x = x + self.dropout(encoder_decoder_attn_output)
        x = self.ln2(x)
        
        ffn_output = self.ffn(x)
        x = x + self.dropout(ffn_output)
        x = self.ln3(x)
        return x

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_dim, max_len=5000, base=10000):
        super().__init__()
        pe = torch.zeros(max_len, emb_dim) # (S, D)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # (S, 1)
        div_term = torch.exp(torch.arange(0, emb_dim, 2).float() * (-math.log(base) / emb_dim)) # (D/2,)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) # (1, S, D)
        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x):
        seq_len = x.shape[1] # (B, S, D)
        return self.pe[:, :seq_len] # (1, S, D)


class RoPE(nn.Module):
    def __init__(self, emb_dim, n_heads, max_len=5000, base=10000):
        super().__init__()
        assert emb_dim % n_heads == 0, "emb_dim must be divisible by n_heads"
        self.base = base
        self.head_dim = emb_dim // n_heads
        self.max_len = max_len  

        inv_freq = 1.0 / (base ** (torch.arange(0, self.head_dim, 2, dtype=torch.float) / self.head_dim))
        pos = torch.arange(max_len, dtype=torch.float).unsqueeze(1)        # (S, 1)
        angle = pos * inv_freq                                             # (S, D/2)

        cos = angle.cos()[None, :, None, :]  # (1, S, 1, D/2)  
        sin = angle.sin()[None, :, None, :]  # (1, S, 1, D/2)

        self.register_buffer('cos', cos, persistent=False)
        self.register_buffer('sin', sin, persistent=False)

    def forward(self, x, start=0):
        # x: [B, S, H, head_dim]
        B, S, H, D = x.shape
        assert D == self.head_dim, "x.shape[-1] must match head_dim"
        assert start + S <= self.max_len, "start + S must be less than max_len"

        cos = self.cos[:, start:start+S].to(x.device, dtype=x.dtype)  # (1, S, 1, D/2)
        sin = self.sin[:, start:start+S].to(x.device, dtype=x.dtype)  # (1, S, 1, D/2)

        x_even, x_odd = x[..., ::2], x[..., 1::2]                     # (B, S, H, D/2)
        x_even_r = x_even * cos - x_odd * sin
        x_odd_r  = x_even * sin + x_odd * cos

        out = torch.empty_like(x)
        out[..., ::2] = x_even_r
        out[..., 1::2] = x_odd_r
        return out


        

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_dim, n_heads, dropout=0.1):
        super().__init__()
        assert emb_dim % n_heads == 0, "emb_dim must be divisible by n_heads"

        self.emb_dim = emb_dim
        self.n_heads = n_heads
        self.head_dim = emb_dim // n_heads

        self.W_Q = nn.Linear(emb_dim, emb_dim)  # (B, S_q, D) 
        self.W_K = nn.Linear(emb_dim, emb_dim)  # (B, S_k, D) 
        self.W_V = nn.Linear(emb_dim, emb_dim)  # (B, S_k, D) 

        self.W_O = nn.Linear(emb_dim, emb_dim)
        self.dropout = nn.Dropout(dropout)

        self.rope = RoPE(emb_dim, n_heads)

    def forward(self, x, kv=None, mask=None):
        # x:  (B, S_q, D)  queries
        # kv: (B, S_k, D)  keys/values source; if None -> self-attention (kv = x)
        B, S_q, D = x.shape
        if kv is None:
            kv = x
        S_k = kv.shape[1]

        # project Q from x, K/V from kv
        q = self.W_Q(x)      # (B, S_q, D)
        k = self.W_K(kv)     # (B, S_k, D)
        v = self.W_V(kv)     # (B, S_k, D)

        # split heads
        q = q.view(B, S_q, self.n_heads, self.head_dim)  # (B, S_q, H, h_dim)
        k = k.view(B, S_k, self.n_heads, self.head_dim)  # (B, S_k, H, h_dim)
        v = v.view(B, S_k, self.n_heads, self.head_dim)  # (B, S_k, H, h_dim)

        q = self.rope(q)
        k = self.rope(k)

        # move heads forward
        q = q.transpose(1, 2)  # (B, H, S_q, h_dim)
        k = k.transpose(1, 2)  # (B, H, S_k, h_dim)
        v = v.transpose(1, 2)  # (B, H, S_k, h_dim)

        # scaled dot-product attention
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)  # (B, H, S_q, S_k)

        if mask is not None:
            if mask.dtype != torch.bool:
                mask = mask != 0

            #   keys(j) → 0  1  2  3
            # q\k(i)↓
            # 0           1  0  0  0
            # 1           1  1  0  0
            # 2           1  1  1  0
            # 3           1  1  1  1
            if mask.ndim == 2 and mask.shape == (S_q, S_k):   # causal/visibility mask
                mask = mask.unsqueeze(0).unsqueeze(0)          # (1, 1, S_q, S_k)
            elif mask.ndim == 2 and mask.shape == (B, S_k):    # key padding mask over K
                mask = mask.unsqueeze(1).unsqueeze(2)          # (B, 1, 1, S_k)
            else:
                raise ValueError(f"Invalid mask shape: {mask.shape}")
            scores = scores.masked_fill(~mask, float("-inf"))  # (B, H, S_q, S_k)

        attn_scores = F.softmax(scores, dim=-1)      # for each query over all keys (B, H, S_q, S_k)
        attn_scores = self.dropout(attn_scores)

        # weighted sum of V
        context_vector = torch.matmul(attn_scores, v)               # (B, H, S_q, h_dim)
        context_vector = context_vector.transpose(1, 2).contiguous()  # (B, S_q, H, h_dim)
        context_vector = context_vector.view(B, S_q, D)             # concat heads -> (B, S_q, D)

        out = self.W_O(context_vector)                             # mix head information -> (B, S_q, D)
        return out


In [ ]:
class MultiHeadAttentionWithCache(nn.Module):
    def __init__(self, emb_dim, n_heads, dropout=0.1):
        super().__init__()
        assert emb_dim % n_heads == 0, "emb_dim must be divisible by n_heads"

        self.emb_dim = emb_dim
        self.n_heads = n_heads
        self.head_dim = emb_dim // n_heads

        self.W_Q = nn.Linear(emb_dim, emb_dim)  
        self.W_K = nn.Linear(emb_dim, emb_dim)  
        self.W_V = nn.Linear(emb_dim, emb_dim)  

        self.W_O = nn.Linear(emb_dim, emb_dim)
        self.dropout = nn.Dropout(dropout)
        self.rope = RoPE(emb_dim, n_heads)

    def forward(self, x, kv=None, past_kv=None, mask=None):
        # prefilling: S_q, S_k > 1
        # decoding: S_q, S_k = 1, past_kv is not None
        B, S_q, D = x.shape
        if kv is None:
            kv = x
        S_k = kv.shape[1]

        q = self.W_Q(x)     
        k = self.W_K(kv)   
        v = self.W_V(kv)     

        q = q.view(B, S_q, self.n_heads, self.head_dim)
        k = k.view(B, S_k, self.n_heads, self.head_dim)
        v = v.view(B, S_k, self.n_heads, self.head_dim)

        q = self.rope(q)
        k = self.rope(k)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        if past_kv is not None:
            k = torch.cat([past_kv[0], k], dim=2)
            v = torch.cat([past_kv[1], v], dim=2)

        S_k = k.shape[2]

        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim) 

        if mask is not None:
            if mask.dtype != torch.bool:
                mask = mask != 0

            if mask.ndim == 2 and mask.shape == (S_q, S_k):   # causal/visibility mask
                mask = mask.unsqueeze(0).unsqueeze(0)          # (1, 1, S_q, S_k)
            elif mask.ndim == 2 and mask.shape == (B, S_k):    # key padding mask over K
                mask = mask.unsqueeze(1).unsqueeze(2)          # (B, 1, 1, S_k)
            else:
                raise ValueError(f"Invalid mask shape: {mask.shape}")
            scores = scores.masked_fill(~mask, float("-inf"))  # (B, H, S_q, S_k)

        attn_scores = F.softmax(scores, dim=-1)      
        attn_scores = self.dropout(attn_scores)

        context_vector = torch.matmul(attn_scores, v)          
        context_vector = context_vector.transpose(1, 2).contiguous() 
        context_vector = context_vector.view(B, S_q, D)          

        out = self.W_O(context_vector)                             
        present_kv = (k, v)
        return out, present_kv


In [ ]:
class MultiQueryAttention(nn.Module):
    def __init__(self, emb_dim, n_heads, dropout=0.1):
        super().__init__()
        assert emb_dim % n_heads == 0, "emb_dim must be divisible by n_heads"

        self.emb_dim = emb_dim
        self.n_heads = n_heads
        self.head_dim = emb_dim // n_heads

        self.W_Q = nn.Linear(emb_dim, emb_dim)  # (B, S_q, D) 
        self.W_K = nn.Linear(emb_dim, self.head_dim)  #  -> (B, S_k, h_dim) 
        self.W_V = nn.Linear(emb_dim, self.head_dim)  # -> (B, S_k, h_dim) 
        self.W_O = nn.Linear(emb_dim, emb_dim)
        self.dropout = nn.Dropout(dropout)

        self.rope = RoPE(emb_dim, n_heads)

    def forward(self, x, kv=None, mask=None):
        B, S_q, D = x.shape
        if kv is None:
            kv = x
        S_k = kv.shape[1]

        q = self.W_Q(x)      # (B, S_q, D)
        k = self.W_K(kv)     # (B, S_k, h_dim)
        v = self.W_V(kv)     # (B, S_k, h_dim)

        q = q.view(B, S_q, self.n_heads, self.head_dim)  # (B, S_q, H, h_dim)
        k = k.view(B, S_k, 1, self.head_dim)  # (B, S_k, 1, h_dim)
        v = v.view(B, S_k, 1, self.head_dim)  # (B, S_k, 1, h_dim)

        q = self.rope(q)
        k = self.rope(k)

        q = q.transpose(1, 2)  # (B, H, S_q, h_dim)
        k = k.transpose(1, 2)  # (B, 1, S_k, h_dim)
        v = v.transpose(1, 2)  # (B, 1, S_k, h_dim)

        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)  # (B, H, S_q, S_k)

        if mask is not None:
            if mask.dtype != torch.bool:
                mask = mask != 0

            if mask.ndim == 2 and mask.shape == (S_q, S_k):   # causal/visibility mask
                mask = mask.unsqueeze(0).unsqueeze(0)          # (1, 1, S_q, S_k)
            elif mask.ndim == 2 and mask.shape == (B, S_k):    # key padding mask over K
                mask = mask.unsqueeze(1).unsqueeze(2)          # (B, 1, 1, S_k)
            else:
                raise ValueError(f"Invalid mask shape: {mask.shape}")
            scores = scores.masked_fill(~mask, float("-inf"))  # (B, H, S_q, S_k)

        attn_scores = F.softmax(scores, dim=-1)      # for each query over all keys (B, H, S_q, S_k)
        attn_scores = self.dropout(attn_scores)

        # weighted sum of V
        context_vector = torch.matmul(attn_scores, v)               # (B, H, S_q, h_dim)
        context_vector = context_vector.transpose(1, 2).contiguous()  # (B, S_q, H, h_dim)
        context_vector = context_vector.view(B, S_q, D)             # concat heads -> (B, S_q, D)

        out = self.W_O(context_vector)                             # mix head information -> (B, S_q, D)
        return out
        
        

In [ ]:
class GroupedQueryAttention(nn.Module):
    def __init__(self, emb_dim, n_heads, kv_groups, dropout=0.1):
        super().__init__()
        assert emb_dim % n_heads == 0, "emb_dim must be divisible by n_heads"
        assert emb_dim % kv_groups == 0, "emb_dim must be divisible by kv_groups"
        assert n_heads % kv_groups == 0, "n_heads must be divisible by kv_groups"

        self.emb_dim = emb_dim
        self.n_heads = n_heads
        self.kv_groups = kv_groups
        self.head_dim = emb_dim // n_heads

        self.W_Q = nn.Linear(emb_dim, emb_dim)  # (B, S_q, D) 
        self.W_K = nn.Linear(emb_dim, self.kv_groups * self.head_dim) 
        self.W_V = nn.Linear(emb_dim, self.kv_groups * self.head_dim) 
        self.W_O = nn.Linear(emb_dim, emb_dim)
        self.dropout = nn.Dropout(dropout)

        self.rope = RoPE(emb_dim, n_heads)

    def forward(self, x, kv=None, mask=None):
        B, S_q, D = x.shape
        if kv is None:
            kv = x
        S_k = kv.shape[1]

        q = self.W_Q(x)      # (B, S_q, D)
        k = self.W_K(kv)     # (B, S_k, h_dim * G)
        v = self.W_V(kv)     # (B, S_k, h_dim * G)

        q = q.view(B, S_q, self.n_heads, self.head_dim)  # (B, S_q, H, h_dim)
        k = k.view(B, S_k, self.kv_groups, self.head_dim)  # (B, S_k, G, h_dim)
        v = v.view(B, S_k, self.kv_groups, self.head_dim)  # (B, S_k, G, h_dim)

        q = self.rope(q)
        k = self.rope(k)

        q = q.transpose(1, 2)  # (B, H, S_q, h_dim)
        k = k.transpose(1, 2)  # (B, G, S_k, h_dim)
        v = v.transpose(1, 2)  # (B, G, S_k, h_dim)

        if self.kv_groups != self.n_heads:
            k = k.repeat(1, self.n_heads // self.kv_groups, 1, 1) #(B, G, S_k, h_dim) -> (B, H, S_k, h_dim)
            v = v.repeat(1, self.n_heads // self.kv_groups, 1, 1)
        
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)  # (B, H, S_q, S_k)

        if mask is not None:
            if mask.dtype != torch.bool:
                mask = mask != 0

            if mask.ndim == 2 and mask.shape == (S_q, S_k):   # causal/visibility mask
                mask = mask.unsqueeze(0).unsqueeze(0)          # (1, 1, S_q, S_k)
            elif mask.ndim == 2 and mask.shape == (B, S_k):    # key padding mask over K
                mask = mask.unsqueeze(1).unsqueeze(2)          # (B, 1, 1, S_k)
            else:
                raise ValueError(f"Invalid mask shape: {mask.shape}")
            scores = scores.masked_fill(~mask, float("-inf"))  # (B, H, S_q, S_k)

        attn_scores = F.softmax(scores, dim=-1)      # for each query over all keys (B, H, S_q, S_k)
        attn_scores = self.dropout(attn_scores)

        # weighted sum of V
        context_vector = torch.matmul(attn_scores, v)               # (B, H, S_q, h_dim)
        context_vector = context_vector.transpose(1, 2).contiguous()  # (B, S_q, H, h_dim)
        context_vector = context_vector.view(B, S_q, D)             # concat heads -> (B, S_q, D)

        out = self.W_O(context_vector)                             # mix head information -> (B, S_q, D)
        return out

        
        

In [ ]:
class MutiheadLatentAttention(nn.Module):
    def __init__(self, emb_dim, down_dim, up_dim, rope_dim, n_heads, dropout=0.1):
        super().__init__()
        assert emb_dim % n_heads == 0, "emb_dim must be divisible by n_heads"

        self.emb_dim = emb_dim
        self.n_heads = n_heads
        self.down_dim = down_dim
        self.up_dim = up_dim
        self.rope_dim = rope_dim
        self.v_head_dim = up_dim // n_heads

        self.down_proj_kv = nn.Linear(emb_dim, down_dim)
        self.down_proj_q = nn.Linear(emb_dim, down_dim)

        self.up_proj_q = nn.Linear(down_dim, up_dim)
        self.up_proj_k = nn.Linear(down_dim, up_dim)
        self.up_proj_v = nn.Linear(down_dim, up_dim)

        self.proj_qr = nn.Linear(down_dim, rope_dim * n_heads)
        self.proj_kr_ = nn.Linear(emb_dim, rope_dim)

        self.rope_q = RoPE(rope_dim * n_heads, n_heads)
        self.rope_k = RoPE(rope_dim, 1)

        self.dropout = nn.Dropout(dropout)
        self.res_dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_heads * self.v_head_dim, emb_dim)

    def forward(self, h, kv=None, mask=None, past_c_kv=None, past_k_t_r=None, start_pos=0):
        B, S_q, D = h.shape

        if kv is None:
            kv = h
        S_k = kv.shape[1]

        c_t_kv = self.down_proj_kv(kv) # (B, S_k, down_dim)
        if past_c_kv is not None:
            c_t_kv = torch.cat([past_c_kv, c_t_kv], dim=1)
        S_k = c_t_kv.shape[1]

        k_t_c = self.up_proj_k(c_t_kv) # (B, S_q, up_dim)
        v_t_c = self.up_proj_v(c_t_kv) # (B, S_q, up_dim)

        c_t_q = self.down_proj_q(h) # (B, S_q, down_dim)
        q_t_c = self.up_proj_q(c_t_q) # (B, S_q, up_dim)

        q_t_r = self.proj_qr(c_t_q) # (B, S_q, rope_dim * n_heads)
        q_t_r = q_t_r.view(B, S_q, self.n_heads, self.rope_dim).transpose(1, 2) # (B, H, S_q, rope_dim)
        q_t_r = self.rope_q(q_t_r, start=start_pos) # (B, H, S_q, rope_dim)

        k_t_r = self.proj_kr_(kv).unsqueeze(1) # (B, 1, S_k, rope_dim)
        k_t_r = self.rope_k(k_t_r, start=start_pos) # (B, 1, S_k, rope_dim)
        if past_k_t_r is not None:
            k_t_r = torch.cat([past_k_t_r, k_t_r], dim=2)


        q_t_c = q_t_c.view(B, S_q, self.n_heads, self.v_head_dim).transpose(1, 2) # (B, H, S_q, v_head_dim)
        q = torch.cat([q_t_c, q_t_r], dim=-1) # (B, H, S_q, rope_dim + v_head_dim)

        k_t_c = k_t_c.view(B, S_k, self.n_heads, self.v_head_dim).transpose(1, 2) # (B, H, S_k, v_head_dim)
        k = torch.cat([k_t_c, k_t_r.expand(B, self.n_heads, S_k, self.rope_dim)], dim=-1) # (B, H, S_k, rope_dim + v_head_dim)

        scores = torch.matmul(q, k.transpose(-2, -1)) / (math.sqrt(self.rope_dim + self.v_head_dim)) # (B, H, S_q, S_k)
        scores = self.dropout(scores)


        if mask is not None:
            if mask.dtype != torch.bool:
                mask = mask != 0

            if mask.ndim == 2 and mask.shape == (S_q, S_k):   # causal/visibility mask
                mask = mask.unsqueeze(0).unsqueeze(0)          # (1, 1, S_q, S_k)
            elif mask.ndim == 2 and mask.shape == (B, S_k):    # key padding mask over K
                mask = mask.unsqueeze(1).unsqueeze(2)          # (B, 1, 1, S_k)
            else:
                raise ValueError(f"Invalid mask shape: {mask.shape}")
            scores = scores.masked_fill(~mask, float("-inf"))  # (B, H, S_q, S_k)
        
        attn_scores = F.softmax(scores, dim=-1) # (B, H, S_q, S_k)
        attn_scores = self.dropout(attn_scores)

        v_t_c = v_t_c.view(B, S_k, self.n_heads, self.v_head_dim).transpose(1, 2) # (B, H, S_k, v_head_dim)
        context_vector = torch.matmul(attn_scores, v_t_c) # (B, H, S_q, v_head_dim)
        context_vector = context_vector.transpose(1, 2).contiguous().view(B, S_q, self.up_dim) # (B, S_q, up_dim)

        out = self.fc(context_vector) # (B, S_q, D)
        out = self.res_dropout(out)
        return out, c_t_kv, k_t_r

In [ ]:
def streaming_softmax_update(logits, v_block, m, l, o):
    """
        logits:  (B, H, S_q, block)    attention logits for this block
        v_block:(B, H, block, h_dim)  value vectors for this block
        m:      (B, H, S_q)           running max over all previous logits
        l:      (B, H, S_q)           running sum of exp(logits - m)
        o:      (B, H, S_q, h_dim)    running sum of exp(logits - m) * V
    """
    # max logits for this block
    m_block = logits.max(dim=-1).values                 # (B, H, S_q)
    # new max over previous blocks + this block
    m_new = torch.maximum(m, m_block)                   # (B, H, S_q)


    """
    sum_{j in P} exp(s_j - m_new)
        = exp(m - m_new) * sum_{j in P} exp(s_j - m)
        = exp(m - m_new) * l

    sum_{i in B} exp(s_i - m_new)
        = sum_i exp_block[i]             # computed directly

    => l_new = exp(m - m_new) * l + sum_i exp_block[i]
    """
    # old exp sum, rescaled to the new max
    exp_old = torch.exp(m - m_new) * l                  # (B, H, S_q)
    # exp values for this block, already shifted by m_new
    exp_block = torch.exp(logits - m_new.unsqueeze(-1)) # (B, H, S_q, block)
    # new denominator
    l_new = exp_old + exp_block.sum(dim=-1)             # (B, H, S_q)


    """
    sum_{j in P} exp(s_j - m_new) v_j
        = exp(m - m_new) * o

    sum_{i in B} exp(s_i - m_new) v_i
        = sum_i exp_block[i] v_i         # matrix product

    => o_new = exp(m - m_new) * o + o_block
    """
    # old numerator, rescaled
    o_old_scaled = torch.exp(m - m_new).unsqueeze(-1) * o   # (B, H, S_q, h_dim)
    # contribution from this block
    o_block = torch.matmul(exp_block, v_block)              # (B, H, S_q, h_dim)
    # new numerator
    o_new = o_old_scaled + o_block                          # (B, H, S_q, h_dim)

    return m_new, l_new, o_new


class FlashMultiHeadAttention(nn.Module):

    def __init__(self, emb_dim, n_heads, dropout=0.1, block_size=128):
        super().__init__()
        assert emb_dim % n_heads == 0, "emb_dim must be divisible by n_heads"

        self.emb_dim = emb_dim
        self.n_heads = n_heads
        self.head_dim = emb_dim // n_heads
        self.block_size = block_size

        self.W_Q = nn.Linear(emb_dim, emb_dim)
        self.W_K = nn.Linear(emb_dim, emb_dim)
        self.W_V = nn.Linear(emb_dim, emb_dim)
        self.W_O = nn.Linear(emb_dim, emb_dim)

        self.dropout = nn.Dropout(dropout)
        self.rope = RoPE(emb_dim, n_heads)

    def forward(self, x, kv=None, mask=None):
        B, S_q, D = x.shape
        if kv is None:
            kv = x
        S_k = kv.shape[1]

        q = self.W_Q(x)      # (B, S_q, D)
        k = self.W_K(kv)     # (B, S_k, D)
        v = self.W_V(kv)     # (B, S_k, D)

        q = q.view(B, S_q, self.n_heads, self.head_dim)
        k = k.view(B, S_k, self.n_heads, self.head_dim)
        v = v.view(B, S_k, self.n_heads, self.head_dim)

        q = self.rope(q)
        k = self.rope(k)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        attn_mask = None
        if mask is not None:
            if mask.dtype != torch.bool:
                mask = mask != 0  # non-zero -> True

            if mask.ndim == 2 and mask.shape == (S_q, S_k):
                attn_mask = mask.unsqueeze(0).unsqueeze(0)     # (1, 1, S_q, S_k)
            elif mask.ndim == 2 and mask.shape == (B, S_k):
                attn_mask = mask.unsqueeze(1).unsqueeze(2)     # (B, 1, 1, S_k)
            else:
                raise ValueError(f"Invalid mask shape: {mask.shape}")


        scale = 1.0 / math.sqrt(head_dim)

        # running max, denominator, numerator
        m = q.new_full((B, H, S_q), float("-inf"))        # (B, H, S_q)
        l = q.new_zeros((B, H, S_q))                     # (B, H, S_q)
        o = q.new_zeros((B, H, S_q, head_dim))           # (B, H, S_q, head_dim)

        for start in range(0, S_k, self.block_size):
            end = min(start + self.block_size, S_k)

            # K/V block: (B, H, block, head_dim)
            k_block = k[:, :, start:end, :]
            v_block = v[:, :, start:end, :]

            # logits for this block: (B, H, S_q, block)
            logits = torch.matmul(q, k_block.transpose(-2, -1)) * scale

            # apply mask on this key range if needed
            if attn_mask is not None:
                # (..., S_k) -> (..., block)
                mask_block = attn_mask[..., start:end]   # broadcastable
                logits = logits.masked_fill(~mask_block, float("-inf"))

            # update streaming softmax state
            m, l, o = streaming_softmax_update(logits, v_block, m, l, o)

        context = o / l.unsqueeze(-1)                     # (B, H, S_q, head_dim)

        context = context.transpose(1, 2).contiguous()    # (B, S_q, H, head_dim)
        context = context.view(B, S_q, D)                 # (B, S_q, D)

        out = self.W_O(context)
        out = self.dropout(out)
        return out
